In [ ]:
import os
import pandas as pd
import numpy as np
import statistics as stat

In [ ]:
path='E:\\HAR Dataset\\WISDM_ar_v1.1'

In [ ]:
os.listdir(path)

In [ ]:
data_path=path+'\\WISDM_ar_v1.1_raw.txt'

In [ ]:
file=open(data_path,'r')

In [ ]:
data_list=[]
for line in file.readlines():
    temp=line.split(';')[0].split(',')
    
    data_list.append(temp)

In [ ]:
for item in data_list:
    if len(item)==7:
        print(item)

In [ ]:
for item in data_list:
    if item[-1]=='':
        print(item)

In [ ]:
for i in range(len(data_list)):
    if len(data_list[i])==7:
        data_list[i]=data_list[i][:-1]

In [ ]:
len(data_list)

In [ ]:
df=pd.DataFrame(data_list,columns=['id','activity','time_stamp','x','y','z'])
df

In [ ]:
df.info()

In [ ]:
data=df.dropna()

In [ ]:
index=data[(data['z']=='\n')].index
index

In [ ]:
x=(data[(data['activity']=='Walking')])
stat.mode(x['z'])

In [ ]:
data['z'].replace('\n','-0.99',inplace=True)

In [ ]:
index=data[(data['z']=='\n')].index
index

In [ ]:
data['z'][index]=stat.mode(x['z'])

In [ ]:
index=data[(data['z']=='\n')].index
index

In [ ]:
data['x']=data['x'].astype('float64')
data['y']=data['y'].astype('float64')
data['z']=data['z'].astype('float64')

In [ ]:
data['activity'].value_counts()

In [ ]:
activity=data['activity'].unique()
activity

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot(ax,x,y,title):
    ax.plot(x,y,'r')
    ax.set_title(title)
    ax.xaxis.set_visible(False)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(x), max(x)])
    ax.grid(True)
    
def plot_data(activity,data):
    fig,axes=plt.subplots(3,figsize=(15, 7), sharex=True)
    fig.suptitle(activity)
    plot(axes[0],data['time_stamp'],data['x'],'X-Axis')
    plot(axes[1],data['time_stamp'],data['y'],'Y-Axis')
    plot(axes[2],data['time_stamp'],data['z'],'Z-Axis')
    plt.show()    

In [ ]:
for item in activity:
    d=data[(data['activity']==item)][:200]
    plot_data(item,d)

In [ ]:
data['activity'].value_counts()

In [ ]:
Walking=data[(data['activity']=='Walking')].iloc[:48395]
Jogging=data[(data['activity']=='Jogging')].iloc[:48395]
Upstairs=data[(data['activity']=='Upstairs')].iloc[:48395]
Downstairs=data[(data['activity']=='Downstairs')].iloc[:48395]
Sitting=data[(data['activity']=='Sitting')].iloc[:48395]
Standing=data[(data['activity']=='Standing')].iloc[:48395]

In [ ]:
balance_data=pd.DataFrame()
balance_data=balance_data.append([Jogging,Walking,Upstairs,Downstairs,Sitting,Standing])

In [ ]:
data_set=balance_data.drop(['id','time_stamp'],axis=1)
data_set

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
data_set['activity']=encoder.fit_transform(data_set['activity'])
encoder.classes_

In [ ]:
x_data=data_set[['x','y','z']]
y_data=data_set['activity']

In [ ]:
y_data=np.asarray(y_data)
y_data=y_data.reshape(-1,)
y_data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_data=scaler.fit_transform(x_data)
x_data

In [ ]:
x_data=pd.DataFrame(x_data,columns=['x','y','z'])

In [ ]:
frequency=20
frame_size=4*frequency
overlapping=2*frequency

In [ ]:
data_frame=[]
label_frame=[]
for i in range(0,(len(x_data)-frame_size),overlapping):
    x=x_data['x'].values[i:i+frame_size]
    y=x_data['y'].values[i:i+frame_size]
    z=x_data['z'].values[i:i+frame_size]
    
    label=stat.mode(y_data[i:i+frame_size])
    arr=np.asarray([x,y,z]).reshape(frame_size,3)
    data_frame.append(arr)
    label_frame.append(label)

In [ ]:
x_data=np.asarray(data_frame)
y_data=np.asarray(label_frame)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.3,random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
import pywt

In [ ]:
x_train_coff=np.ndarray(shape=(x_train.shape[0],80,80,3))
scale = range(1,81)
wavelet = 'gaus5'
for i in range(x_train.shape[0]):   
    for j in range(0,3):
        sig=x_train[i,:,j]
        coeff,freq=pywt.cwt(sig,scale,wavelet)
        coeff_=coeff[:,:80]
        x_train_coff[i,:,:,j]=coeff_

In [ ]:
x_test_coff=np.ndarray(shape=(x_test.shape[0],80,80,3))
scale = range(1,81)
wavelet = 'gaus5'
for i in range(x_test.shape[0]):   
    for j in range(0,3):
        sig=x_test[i,:,j]
        coeff,freq=pywt.cwt(sig,scale,wavelet)
        coeff_=coeff[:,:80]
        x_test_coff[i,:,:,j]=coeff_

In [ ]:
np.savez('...\\HAR\\Preprocessed Data\\Train.npz',x_train=x_train_coff,y_train=y_train)
np.savez('...\\HAR\\Preprocessed Data\\Test.npz',x_test=x_test_coff,y_test=y_test)